# A Comparison of Regressions for Multivariate Count Data

In [1]:
import matplotlib
%matplotlib nbagg
from matplotlib import pyplot

In [2]:
%reload_ext rpy2.ipython

In [3]:
%%R
library(MGLM)
library(bivpois)

In [4]:
from statiskit import (linalg,
                       core,
                       glm)

In [5]:
%%R
data('ex4.ita91')
ita91 = ex4.ita91
ita91['X'] = apply(ita91[,c(1,2)], 1, sum)
write.csv(ita91[c(3,4,5)], 'ita91.csv', row.names=F)
ita91 = ita91[,-c(5)]

In [9]:
data = core.read_csv('ita91.csv', sep=',', header=True, ipdb=True)

> /home/pfernique/.miniconda/envs/statiskit-dev/lib/python2.7/site-packages/statiskit/core/io.py(130)from_list()
-> dataframe = MultivariateDataFrame()
(Pdb) cont


In [10]:
dst = core.selection(data=data.X,
                     estimators=[core.poisson_estimation('ml'),
                                 core.binomial_estimation('ml', force=True),
                                 core.negative_binomial_estimation('ml'),
                                 core.shifted_estimation(core.outcome_type.DISCRETE,
                                                         shift=-1,
                                                         estimator=core.logarithmic_estimation('ml')),
                                 core.frequency_estimation(core.outcome_type.DISCRETE)])
dst.scores

[-565.3220940311769, -582.4311016773115, -565.9357309449964, -685.4209196019883, -576.2948158630275]

In [15]:
dst.estimations[2].estimated

NB(11.7677159885, 0.161781370058)

In [12]:
reg = core.selection(data=data.extract(response=-1),
                     estimators=[glm.poisson_estimation('Fisher'),
                                 glm.binomial_estimation('SA')])
reg.scores

[-629.3253069941982, -632.3068952587911]

In [6]:
%%R
N = ita91[,c(1,2)]
X = ita91[,c(3,4)]
X = as.matrix(X[apply(N, 1, sum) > 0,])
N = as.matrix(N[apply(N, 1, sum) > 0,])

In [8]:
%R print(MGLMfit(N, dist="MN"))

          estimate         SE
alpha_g1 0.5989072 0.02993883
alpha_g2 0.4010928 0.02993883

Distribution: Multinomial
Log-likelihood: -284.2943
BIC: 574.1796
AIC: 570.5886
LRT test p value: 
Iterations: 


In [9]:
%R print(MGLMreg(N ~ X, dist="MN"))


Error in `[[<-.data.frame`(`*tmp*`, i, value = c(2L, 3L, 4L, 5L, 6L, 7L,  : 
  replacement has 536 rows, data has 268


/home/pfernique/.miniconda/envs/FPD17/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in `[[<-.data.frame`(`*tmp*`, i, value = c(2L, 3L, 4L, 5L, 6L, 7L,  : 
  replacement has 536 rows, data has 268

  warnings.warn(x, RRuntimeWarning)


In [12]:
%R print(glm(N ~ X, family="binomial"))


Error in `[[<-.data.frame`(`*tmp*`, i, value = c(2L, 3L, 4L, 5L, 6L, 7L,  : 
  replacement has 536 rows, data has 268


In [15]:
%R print(MGLMfit(N, dist="DM"))

         estimate       SE
alpha_g1 93.66414 443.3854
alpha_g2 65.04395 308.1834

Distribution: Dirichlet Multinomial
Log-likelihood: -284.1546
BIC: 579.4912
AIC: 572.3092
LRT test p value: 0.597
Iterations: 20


In [16]:
%R print(MGLMreg(N ~ X, dist="DM"))


Error in `[[<-.data.frame`(`*tmp*`, i, value = c(2L, 3L, 4L, 5L, 6L, 7L,  : 
  replacement has 536 rows, data has 268


In [53]:
%R print(MGLMreg(N ~ X, dist="NegMN"))


Error in `[[<-.data.frame`(`*tmp*`, i, value = c(2L, 3L, 4L, 5L, 6L, 7L,  : 
  replacement has 536 rows, data has 268


In [51]:
%R model = lm.bp(g1~team1+team2, g2~team1+team2, l1l2=~team1+team2, data=ita91)
%R print(model)

           iter         loglike Rel.Dif.loglike 
           1.00         -759.82            1.76 
           iter         loglike Rel.Dif.loglike 
      2.000e+00      -7.559e+02       5.182e-03 
           iter         loglike Rel.Dif.loglike 
      3.000e+00      -7.555e+02       4.531e-04 
           iter         loglike Rel.Dif.loglike 
      4.000e+00      -7.552e+02       4.379e-04 
           iter         loglike Rel.Dif.loglike 
      5.000e+00      -7.549e+02       4.371e-04 
           iter         loglike Rel.Dif.loglike 
      6.000e+00      -7.546e+02       4.337e-04 
           iter         loglike Rel.Dif.loglike 
      7.000e+00      -7.542e+02       4.269e-04 
           iter         loglike Rel.Dif.loglike 
      8.000e+00      -7.539e+02       4.168e-04 
           iter         loglike Rel.Dif.loglike 
      9.000e+00      -7.536e+02       4.037e-04 
           iter         loglike Rel.Dif.loglike 
      1.000e+01      -7.533e+02       3.880e-04 
           iter     


Call:
lm.bp(l1 = g1 ~ team1 + team2, l2 = g2 ~ team1 + team2, l1l2 = ~team1 + 
    team2, l3 = ~1, data = ita91, common.intercept = FALSE, zeroL3 = FALSE, 
    maxit = 300, pres = 1e-08, verbose = FALSE)

Coefficients:
    (l1):(Intercept)  (l1):team1Atalanta    (l1):team1Bari        
            -0.24829               0.16367               0.05607  
(l1):team1Cagliari    (l1):team1Cremonese   (l1):team1Fiorentina  
             0.61254               0.37469               0.96217  
(l1):team1Foggia      (l1):team1Genoa       (l1):team1Inter       
             1.20051               0.69340              -0.16676  
(l1):team1Juventus    (l1):team1Lazio       (l1):team1Milan       
             0.95972               1.01397               1.53802  
(l1):team1Napoli      (l1):team1Parma       (l1):team1Roma        
             1.26216               0.56203               0.56769  
(l1):team1Sampdoria   (l1):team1Torino      (l1):team1Verona      
             1.00913               0.81364 

In [50]:
%R print(model$BIC)

Saturated   BivPois 
 4899.038  1956.104 


In [15]:
%%R
data('rnaseq')
rnaseq['totalReads'] = log(rnaseq['totalReads'])
rnaseq['X'] = apply(rnaseq[,c(1,2,3,4,5,6)], 1, sum)
write.table(rnaseq[c(7,8,9,10,11)], 'rnaseq.csv', row.names=F)
rnaseq = rnaseq[,-c(11)]

In [16]:
data = core.read_csv('rnaseq.csv', header=True)

In [ ]:
dst = core.selection(data=data.X,
                     estimators=[core.poisson_estimation('ml'),
                                 core.binomial_estimation('ml', force=True),
                                 core.negative_binomial_estimation('ml'),
                                 core.logarithmic_estimation('ml'),
                                 core.frequency_estimation(core.outcome_type.DISCRETE)])
dst.scores

[-6537.059114151809, -13223.754412953454, -1339.360020084458, -1753.9600794771827, -1449.3337676626065]

In [ ]:
reg = core.selection(data=data.extract(response=-1),
                     estimators=[glm.poisson_estimation('Fisher'),
                                 glm.binomial_estimation('SA'),
                                 glm.negative_binomial_estimation('X2')])
reg.scores

In [ ]:
%%R
library(MGLM)
library(bivpois)
N = as.matrix(rnaseq[,c(1,2,3,4,5,6)])
X = as.matrix(rnaseq[,c(7,8,9,10)])

In [ ]:
%R print(MGLMfit(N, dist="MN"))

In [ ]:
%R print(MGLMreg(N ~ X, dist="MN"))

In [ ]:
%R print(MGLMfit(N, dist="DM"))

In [ ]:
%R print(MGLMreg(N ~ X, dist="DM"))

In [ ]:
%R print(MGLMfit(N, dist="GDM"))

In [ ]:
%R print(MGLMreg(N ~ X, dist="GDM"))

In [ ]:
%R print(MGLMreg(N ~ X, dist="NegMN"))